In [1]:
import time
import datetime
import pytz
import calendar
import itertools
import pandas as pd

In [2]:
# getting the utils file here
import os, sys
sys.path.insert(0, '../')
import utils3 as utils

In [3]:
import datetime
import sys

import networkx as nx
import numpy as np
import os
import pandas as pd
import plotly.offline as py
import pytz

sys.path.insert(0, '../')
# import utils
# from utils import plotly_figure

import itertools

In [19]:
class OptimizerParent:
    def __init__(self, building, zones, start, end, window):
        
        self.start = start
        self.unix_start = start.timestamp() * 1e9
        self.end = end
        self.unix_end = end.timestamp() * 1e9
        self.window = window  # timedelta
        
        self.building = building
        self.zones = zones 
        
        # Documentation: All data here is in timeseries starting exactly at start and every step corresponds to one 
        # interval. The end is not inclusive.
        
        # temperature band
        temperature_band_channel = grpc.insecure_channel(TEMPERATURE_BAND_ADDRESS)
        temperature_band_stub = schedules_pb2_grpc.SchedulesStub(temperature_band_channel)
        self.comfortband = {iter_zone: get_comfortband(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                              for iter_zone in self.zones}
        self.do_not_exceed = {iter_zone: get_do_not_exceed(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                              for iter_zone in self.zones}
        
        # occupancy
        occupancy_channel = grpc.insecure_channel(OCCUPANCY_ADDRESS)
        occupancy_stub = occupancy_pb2_grpc.OccupancyStub(occupancy_channel)
        self.occupancy = {iter_zone: get_occupancy(occupancy_stub, self.building, iter_zone, self.start, self.end, self.window)
                         for iter_zone in self.zones}
        
        # outdoor temperatures
        outdoor_historic_channel = grpc.insecure_channel(OUTSIDE_HISTORICAL)
        outdoor_historic_stub = outdoor_temperature_historical_pb2_grpc.OutdoorTemperatureStub(outdoor_historic_channel)
        outdoor_prediction_channel = grpc.insecure_channel(OUTSIDE_PREDICTION)
        outdoor_prediction_stub = outdoor_temperature_prediction_pb2_grpc.OutdoorTemperatureStub(outdoor_prediction_channel)

#         self.outdoor_temperatures = get_outside_temperature(
#             outdoor_historic_stub, outdoor_prediction_stub, self.building, self.start, self.end, self.window)

        # discomfort channel 
        discomfort_channel = grpc.insecure_channel(DISCOMFORT_ADDRESS)
        self.discomfort_stub = discomfort_pb2_grpc.DiscomfortStub(discomfort_channel)
        
        # HVAC Consumption
        hvac_consumption_channel = grpc.insecure_channel(HVAC_CONSUMPTION_ADDRESS)
        hvac_consumption_stub = hvac_consumption_pb2_grpc.ConsumptionHVACStub(hvac_consumption_channel)        
        self.hvac_consumption = {iter_zone: get_hvac_consumption(hvac_consumption_stub, building, iter_zone) 
                                 for iter_zone in self.zones}
        
        # TODO Prices
        


In [112]:

# TODO Demand charges right now assume constant demand charge throughout interval. should be easy to extend
# TODO but need to keep in mind that we need to then store the cost of demand charge and not the consumption
# TODO in the graph, since we actually only want to minimize cost and not consumption.


class Node:
    """
    # this is a Node of the graph for the shortest path
    """

    def __init__(self, temperatures, timestep):
        self.temperatures = temperatures
        self.timestep = timestep

    def __hash__(self):
        return hash((' '.join(str(e) for e in self.temperatures), self.timestep))

    def __eq__(self, other):
        return isinstance(other, self.__class__) \
               and self.temperatures == other.temperatures \
               and self.timestep == other.timestep

    def __repr__(self):
        return "{0}-{1}".format(self.timestep, self.temperatures)


class MPC(OptimizerParent):
    """MPC Optimizer. 
    No Demand Charges and Two Stage actions implemented."""
    
    def __init__(self, building, zones, start, end, window, lambda_val,
                 root=Node([75], 0), debug=False):
        """
        initialize instance variables
        
        :param building: (str) building name
        :param zones: [str] zone names
        :param start: (datetime timezone aware) 
        :param end: (datetime timezone aware) 
        :param window: (str) the interval in which to split the data.
        :param lambda_val: (float) lambda value for opjective function
        :param root: (Node) the node at which to start

        """
        super().__init__(building, zones, start, end, window)

        self.root = root
        self.lambda_val = lambda_val
        self.debug = debug

        self.g = nx.DiGraph()  # [TODO:Changed to MultiDiGraph... FIX print]
        
    def safety_check(self, node):
        for iter_zone in self.zones:
            curr_temperature = node.temperatures[iter_zone]
            curr_safety = self.do_not_exceed[iter_zone].iloc[node.timestep]
            if not (curr_safety["t_low"] <= curr_temperature <= curr_safety["t_high"]):
                return False
        return True
    
    def timestep_to_datetime(self, timestep):
        return start + timestep*datetime.timedelta(seconds=utils.get_window_in_sec(self.window))

    # the shortest path algorithm
    def shortest_path(self, root):
        """
        Creates the graph using DFS and calculates the shortest path
    
        :param root: node being examined right now and needs to be added to graph. 
        
        :return root Node if root added else return None. 
        
        """
        
        if root is None:
            return None
        
        if root in self.g:
            return root
                
        # stop if node is past predictive horizon
        if self.timestep_to_datetime(root.timestep) >= self.end:
            self.g.add_node(root, objective_cost=0, best_action=None, best_successor=None) # no cost as leaf node
            return root
        
        # check if valid node
        if not self.safety_check(root):
            return None
        
        self.g.add_node(root, objective_cost=np.inf, best_action=None, best_successor=None)

        # creating children, adding corresponding edge and updating root's objective cost
        for action in itertools.product([utils.NO_ACTION, utils.HEATING_ACTION, utils.COOLING_ACTION], 
                                       repeat=len(self.zones)):
                
            # TODO Compute temperatures properly
            temperatures = {}
            for i in range(len(self.zones)):
                 temperatures[self.zones[i]] = root.temperatures[self.zones[i]] + \
                                                    1 * (action[i]==1) - 1*(action[i]==2)

            child_node = Node(
                temperatures=temperatures,
                timestep=root.timestep + 1
                )

            child_node = self.shortest_path(child_node)
            if child_node is None:
                continue

            # get discomfort across edge
            discomfort = {}
            for iter_zone in self.zones:
                curr_comfortband = self.comfortband[iter_zone].iloc[root.timestep]
                curr_occupancy = self.occupancy[iter_zone].iloc[root.timestep]
                average_edge_temperature = (root.temperatures[iter_zone] + child_node.temperatures[iter_zone])/2.

                discomfort[iter_zone] = get_discomfort(
                    self.discomfort_stub, self.building, average_edge_temperature,
                    curr_comfortband["t_low"], curr_comfortband["t_high"], 
                    curr_occupancy)

            # Get consumption across edge
            price = 1  # self.prices.iloc[root.timestep] TODO also add right unit conversion, and duration
            consumption_cost = {self.zones[i]: price * self.hvac_consumption[self.zones[i]][action[i]] 
                               for i in range(len(self.zones))}

            # add edge 
            self.g.add_edge(root, child_node, action=action, discomfort=discomfort, consumption_cost=consumption_cost)

            # update root node to contain the best child.
            total_edge_cost = ((1 - self.lambda_val) * (sum(consumption_cost.values()))) + (
                self.lambda_val * (sum(discomfort.values())))
            
            objective_cost = self.g.node[child_node]["objective_cost"] + total_edge_cost

            if objective_cost < self.g.node[root]["objective_cost"]:
                self.g.node[root]["objective_cost"] = objective_cost
                self.g.node[root]["best_action"] = action
                self.g.node[root]["best_successor"] = child_node
                
        return root
 
    def reconstruct_path(self, root):
        """
        Util function that reconstructs the best action path
        Parameters
        ----------
        graph : networkx graph

        Returns
        -------
        List
        """
        graph = self.g 
        
        if root not in self.g:
            raise Exception("Root does not exist in MPC graph.")

        
        path = [root]

        while graph.node[root]['best_successor'] is not None:
            root = graph.node[root]['best_successor']
            path.append(root)

        return path
    
#     def g_plot(self, zone):
#         try:
#             os.remove('mpc_graph_' + zone + '.html')
#         except OSError:
#             pass

#         fig = plotly_figure(self.advise_unit.g, path=self.path)
#         py.plot(fig, filename='mpc_graph_' + zone + '.html', auto_open=False)


In [121]:

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
start =  end - datetime.timedelta(hours=4)
print(start)
print(start.timestamp())
bldg = "avenal-animal-shelter"
zone = "HVAC_Zone_Shelter_Corridor"
window = "15m"

# print(get_do_not_exceed(temperature_band_stub, bldg, zone, start, end, window))
# print(get_discomfort(discomfort_stub, bldg, 78, 65, 75, 1))
# print(get_hvac_consumption(hvac_consumption_stub, bldg, zone))
op = MPC(bldg, [zone], start, end, window, 0.995)

2019-02-03 02:16:22.056157+00:00
1549160182.056157


In [122]:
root = Node({zone: 85}, 0)
op.occupancy[zone][:] = 1

In [123]:
op.occupancy

{'HVAC_Zone_Shelter_Corridor': 2019-02-03 02:16:22.056157+00:00    1.0
 2019-02-03 02:31:22.056157+00:00    1.0
 2019-02-03 02:46:22.056157+00:00    1.0
 2019-02-03 03:01:22.056157+00:00    1.0
 2019-02-03 03:16:22.056157+00:00    1.0
 2019-02-03 03:31:22.056157+00:00    1.0
 2019-02-03 03:46:22.056157+00:00    1.0
 2019-02-03 04:01:22.056157+00:00    1.0
 2019-02-03 04:16:22.056157+00:00    1.0
 2019-02-03 04:31:22.056157+00:00    1.0
 2019-02-03 04:46:22.056157+00:00    1.0
 2019-02-03 05:01:22.056157+00:00    1.0
 2019-02-03 05:16:22.056157+00:00    1.0
 2019-02-03 05:31:22.056157+00:00    1.0
 2019-02-03 05:46:22.056157+00:00    1.0
 2019-02-03 06:01:22.056157+00:00    1.0
 Freq: 900S, dtype: float64}

In [124]:
root = op.shortest_path(root)
print(root)
# op.g.node[root]

{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.5}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor': 0.0}
{'HVAC_Zone_Shelter_Corridor

In [125]:
op.g.node[root]["best_successor"]

1-{'HVAC_Zone_Shelter_Corridor': 84}

In [ ]:
op.safety_check(root)

In [156]:
op.timestep_to_time(2)

datetime.datetime(2019, 2, 3, 2, 5, 55, 611324, tzinfo=<UTC>)

In [127]:
op.reconstruct_path(root)

[0-{'HVAC_Zone_Shelter_Corridor': 85},
 1-{'HVAC_Zone_Shelter_Corridor': 84},
 2-{'HVAC_Zone_Shelter_Corridor': 83},
 3-{'HVAC_Zone_Shelter_Corridor': 82},
 4-{'HVAC_Zone_Shelter_Corridor': 81},
 5-{'HVAC_Zone_Shelter_Corridor': 80},
 6-{'HVAC_Zone_Shelter_Corridor': 79},
 7-{'HVAC_Zone_Shelter_Corridor': 78},
 8-{'HVAC_Zone_Shelter_Corridor': 78},
 9-{'HVAC_Zone_Shelter_Corridor': 78},
 10-{'HVAC_Zone_Shelter_Corridor': 78},
 11-{'HVAC_Zone_Shelter_Corridor': 78},
 12-{'HVAC_Zone_Shelter_Corridor': 78},
 13-{'HVAC_Zone_Shelter_Corridor': 78},
 14-{'HVAC_Zone_Shelter_Corridor': 78},
 15-{'HVAC_Zone_Shelter_Corridor': 78},
 16-{'HVAC_Zone_Shelter_Corridor': 78}]

import sys

In [5]:
import sys
sys.path.append("../microservices_wrapper")
import microservices_wrapper as mw

In [6]:
temperature_band_stub = mw.get_temperature_band_stub()

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
start =  end - datetime.timedelta(hours=4)
print(start)
print(start.timestamp())
bldg = "avenal-animal-shelter"
zone = "HVAC_Zone_Shelter_Corridor"
window = "15m"

mw.get_do_not_exceed(temperature_band_stub, bldg, zone, start, end, window)

2019-02-07 03:23:11.038332+00:00
1549509791.038332


_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connect Failed"
	debug_error_string = "{"created":"@1549524191.040239000","description":"Failed to create subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":2721,"referenced_errors":[{"created":"@1549524191.040237000","description":"Pick Cancelled","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":241,"referenced_errors":[{"created":"@1549524191.040219000","description":"Connect Failed","file":"src/core/ext/filters/client_channel/subchannel.cc","file_line":689,"grpc_status":14,"referenced_errors":[{"created":"@1549524191.040210000","description":"Failed to connect to remote host: OS Error","errno":61,"file":"src/core/lib/iomgr/tcp_client_posix.cc","file_line":205,"os_error":"Connection refused","syscall":"connect","target_address":"ipv4:127.0.0.1:50055"}]}]}]}"
>

In [9]:
utils.get_zones("avenal-movie-theatre")

['HVAC_Zone_Room_A',
 'HVAC_Zone_Main_Hallway',
 'HVAC_Zone_Pegasus_Hall',
 'HVAC_Zone_Room_D',
 'HVAC_Zone_Theater_1',
 'HVAC_Zone_Lobby',
 'HVAC_Zone_Back_Hallway',
 'HVAC_Zone_Theater_2']

In [14]:
import sys
sys.path.append("../microservices_wrapper/")
import microservices_wrapper as mw 


In [16]:
outdoor_stub = mw.get_outdoor_historic_stub()
occ_stub = mw.get_occupancy_stub()
temperature_band_stub = mw.get_temperature_band_stub()
indoor_prediction_stub = mw.get_indoor_temperature_prediction_stub()

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
start =  end - datetime.timedelta(days=100)
print(start)
print(start.timestamp())
bldg = "avenal-animal-shelter"
zone = "HVAC_Zone_Shelter_Corridor"
window = "5m"

# mw.get_outdoor_temperature_historic(outdoor_stub, bldg, start, end, window)
# mw.get_occupancy(occ_stub, bldg, zone, start, end, window)
# mw.get_comfortband(temperature_band_stub, bldg, zone, start, end, window)
mw.get_indoor_temperature_prediction(indoor_prediction_stub, bldg, zone, start, 2, 70, 80, {}, 1)

2018-10-30 19:24:17.557010+00:00
1540927457.55701


2018-10-30 19:24:17.557010+00:00    65.999705
2018-10-30 19:29:17.557010+00:00    65.999705
2018-10-30 19:34:17.557010+00:00    65.999705
2018-10-30 19:39:17.557010+00:00    65.999705
2018-10-30 19:44:17.557010+00:00    65.999705
2018-10-30 19:49:17.557010+00:00    65.999705
2018-10-30 19:54:17.557010+00:00    66.166370
2018-10-30 19:59:17.557010+00:00    66.333035
2018-10-30 20:04:17.557010+00:00    66.499700
2018-10-30 20:09:17.557010+00:00    66.666500
2018-10-30 20:14:17.557010+00:00    66.833300
2018-10-30 20:19:17.557010+00:00    67.000100
2018-10-30 20:24:17.557010+00:00    67.000100
2018-10-30 20:29:17.557010+00:00    67.000100
2018-10-30 20:34:17.557010+00:00    67.000100
2018-10-30 20:39:17.557010+00:00    67.166771
2018-10-30 20:44:17.557010+00:00    67.333442
2018-10-30 20:49:17.557010+00:00    67.500113
2018-10-30 20:54:17.557010+00:00    67.500113
2018-10-30 20:59:17.557010+00:00    67.500113
2018-10-30 21:04:17.557010+00:00    67.500113
2018-10-30 21:09:17.557010+00:00  

In [ ]:
building="ciee",
                                                                              zone="HVAC_Zone_Northzone",
                                                                              current_time=start,
                                                                              indoor_temperature=70,
                                                                              outside_temperature=80,
                                                                              action=0,
                                                                              zone_temperatures={
                                                                                  "HVAC_Zone_Eastzone": 75,
                                                                                  "HVAC_Zone_Southzone": 60,
                                                                                  "HVAC_Zone_Centralzone": 80
                                                                              },
                                                                                temperature_unit="F",
                                                                              occupancy=1))

In [37]:
pd.Timedelta(42, unit='ns').nanoseconds

42

In [42]:
int("1.0")

ValueError: invalid literal for int() with base 10: '1.0'

In [17]:
def round_to_midnight(date, interval):
    """Takes the date and brings the time as close as possible to midnight such that
    (date - new_time).total_seconds % interval == 0.
    :param date: (datetime) date to use for rounding
    :param interval int:seconds.
    :return datetime new_time"""

    dt = datetime.timedelta(seconds=interval)
    curr_day = date.day
    while (date - dt).day == curr_day:
        date -= dt
    return date

In [20]:
curr_time = datetime.datetime.utcnow()
print(curr_time)
round_to_midnight(curr_time, 2 * 60 * 60)

2019-02-08 00:22:57.543441


datetime.datetime(2019, 2, 8, 0, 22, 57, 543441)